In [15]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Lambda, Input, Dense, Conv2D, BatchNormalization, Reshape, \
                        MaxPool2D, GlobalAveragePooling2D, LeakyReLU, UpSampling2D, Flatten, Conv2DTranspose 
from keras.models import Model
from keras.losses import mean_squared_error, binary_crossentropy, categorical_crossentropy
from keras.utils import plot_model
from keras import backend as K

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import save_img
from keras.callbacks import ModelCheckpoint
#import keras

import numpy as np
import matplotlib.pyplot as plt
import os
import pydot
from PIL import Image
import tensorflow as tf
import random 

# UNCOMMENT TO RUN ON THE GPU(S)
# num_gpus = 2
# config = tf.compat.v1.ConfigProto(device_count = {'GPU':num_gpus})
# sess = tf.compat.v1.Session(config=config)
# tf.compat.v1.keras.backend.set_session(sess)


In [16]:
'''
This box is to only test and view some of the files in the dataset.
The actual dataset generator the the class DataGenerator in the box below.
'''
import numpy as np
import os
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

dataDir = "/Volumes/HardDrive/Fonts/datasetOnlyAlpha/"
x_train = []

for i, file in enumerate(os.listdir(dataDir)[:400]):
    if file[0:2] == "._":
        file = file[2:]
    fontImg = load_img(dataDir+file).convert('L')
    fontImgArr = img_to_array(fontImg)
    x_train.append(fontImgArr)

x_train = np.asarray(x_train)


In [17]:
import random 

partition = {'train':[],'validation':[]}
dataDir = "/Volumes/HardDrive/Fonts/datasetOnlyAlpha/"

allImg = os.listdir(dataDir)
random.shuffle(allImg)

partition['train'] = allImg[: int(len(allImg) * .80)]
partition['validation'] = allImg[int(len(allImg) * .80): int(len(allImg))]

print(len(partition['validation']))

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels=[], batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            dataDir = '/Volumes/HardDrive/Fonts/datasetOnlyAlpha/'
            file = ID
            if file[0:2] == "._":
                file = file[2:]
                
            fontImg = load_img(dataDir+file).convert('L')
            fontImgArr = img_to_array(fontImg)
            fontImgArr = fontImgArr.astype('float32') / 255
            X[i,] = fontImgArr

            # Store class
            y[i,] = fontImgArr

        return X, y

11816


In [18]:
'''Example of VAE on MNIST dataset using MLP
The VAE has a modular design. The encoder, decoder and VAE
are 3 models that share weights. After training the VAE model,
the encoder can be used to generate latent vectors.
The decoder can be used to generate MNIST digits by sampling the
latent vector from a Gaussian distribution with mean = 0 and std = 1.
# Reference
[1] Kingma, Diederik P., and Max Welling.
"Auto-Encoding Variational Bayes."
https://arxiv.org/abs/1312.6114
'''

# reparameterization trick
# instead of sampling from Q(z|X), sample epsilon = N(0,I)
# z = z_mean + sqrt(var) * epsilon
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [19]:
image_size = x_train.shape[1]

# network parameters
input_shape = (image_size, image_size, num_channels)
latent_dim = 100

In [20]:
# # =================
# # Encoder Model
# # =================

inputs      = Input(shape=input_shape, name='encoder_input')
cx      = Conv2D(filters=8, kernel_size=3, strides=2, padding='same', activation='relu')(inputs)
cx      = BatchNormalization()(cx)
cx      = Conv2D(filters=16, kernel_size=3, strides=2, padding='same', activation='relu')(cx)
cx      = BatchNormalization()(cx)
x       = Flatten()(cx)
x       = Dense(20, activation='relu')(x)
x       = BatchNormalization()(x)
z_mean      = Dense(latent_dim, name='latent_mu')(x)
z_log_var   = Dense(latent_dim, name='latent_sigma')(x)
z       = Lambda(sampling, output_shape=(latent_dim, ), name='z')([z_mean, z_log_var])

conv_shape = K.int_shape(cx)

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
# plot_model(encoder, to_file='fonts_vae_encoder.png', show_shapes=True)

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 256, 256, 8)  80          encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 256, 256, 8)  32          conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 128, 128, 16) 1168        batch_normalization_19[0][0]     
____________________________________________________________________________________________

In [21]:
# =================
# Decoder Model
# =================

# Definition
latent_inputs   = Input(shape=(latent_dim, ), name='decoder_input')
x     = Dense(conv_shape[1] * conv_shape[2] * conv_shape[3], activation='relu')(latent_inputs)
x     = BatchNormalization()(x)
x     = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(x)
cx    = Conv2DTranspose(filters=16, kernel_size=3, strides=2, padding='same', activation='relu')(x)
cx    = BatchNormalization()(cx)
cx    = Conv2DTranspose(filters=8, kernel_size=3, strides=2, padding='same',  activation='relu')(cx)
cx    = BatchNormalization()(cx)
outputs     = Conv2DTranspose(filters=num_channels, kernel_size=3, activation='sigmoid', padding='same', name='decoder_output')(cx)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
# plot_model(decoder, to_file='fonts_vae_decoder.png', show_shapes=True)

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 262144)            26476544  
_________________________________________________________________
batch_normalization_22 (Batc (None, 262144)            1048576   
_________________________________________________________________
reshape_4 (Reshape)          (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 256, 256, 16)      2320      
_________________________________________________________________
batch_normalization_23 (Batc (None, 256, 256, 16)      64        
_________________________________________________________________
conv2d_transpose_8 (Conv2DTr (None, 512, 512, 8)       1160

In [22]:
# instantiate VAE model
out = decoder(encoder(inputs)[2])  # index 2 is the sampled tensor from the mean and std
vae = Model(inputs, out, name='vae_fonts')

In [23]:
# for later use when we save the weights
use_weights = False

# Define loss
def kl_reconstruction_loss(true, pred):
    # Reconstruction loss
    reconstruction_loss = binary_crossentropy(K.flatten(true), K.flatten(pred)) * image_size * image_size
    # KL divergence loss
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    # Total loss = 50% rec + 50% KL divergence loss
    return K.mean(reconstruction_loss + kl_loss)

vae.compile(optimizer='adam', loss=kl_reconstruction_loss, metrics=['accuracy'])
vae.summary()

epochs = 50
batch_size = 16
num_classes = 1
num_channels = 1
shuffle = True
# Parameters
params = {'dim': (image_size,image_size),
          'batch_size': batch_size,
          'n_classes': num_classes,
          'n_channels': num_channels,
          'shuffle': shuffle}

# Generators
training_generator = DataGenerator(partition['train'], **params)
validation_generator = DataGenerator(partition['validation'], **params)

print(len(partition['train']))

checkpointer = ModelCheckpoint(filepath='/Users/yigitatay/Desktop/FONT_GENERATOR/checkpoints/bestweights.hdf5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer]

steps_per_epoch = len(partition['train'])/batch_size

# Train model on dataset
vae.fit_generator(generator=training_generator,
                  validation_data=validation_generator,
                  use_multiprocessing=False,
                  workers=8, epochs=epochs, steps_per_epoch=steps_per_epoch,callbacks=callbacks_list)

# uncomment if you want to save the weights
vae.save_weights('/Users/yigitatay/Desktop/FONT_GENERATOR/checkpoints/vae_fonts.h5')

Model: "vae_fonts"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 512, 512, 1)       0         
_________________________________________________________________
encoder (Model)              [(None, 100), (None, 100) 5248524   
_________________________________________________________________
decoder (Model)              (None, 512, 512, 1)       27528769  
Total params: 32,777,293
Trainable params: 32,252,869
Non-trainable params: 524,424
_________________________________________________________________
47263
Epoch 1/50
   4/2953 [..............................] - ETA: 3:59:55 - loss: 227095.8555 - accuracy: 0.4772

KeyboardInterrupt: 

In [61]:
## TEST RECONSTRUCTION ON A TRAINING EXAMPLE
train_ex = x_train[270]
train_ex = train_ex.astype('float32') / 255
train_ex = np.reshape(train_ex, [1, 512, 512, 1])
result = vae.predict(train_ex)

In [62]:
## SAVE THE RESULT
result = np.reshape(result, [512, 512])
result = result * 255
result = result.astype('uint8')
img = Image.fromarray(result, 'L')
img.save("example_training_result.png")

In [24]:
# to see the number of available GPUs
tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.is_built_with_cuda())

(512, 512, 3)
Num GPUs Available:  0
False
